# Import Data and data Processing

In [ ]:
import pandas as pd

In [ ]:
precipitation=pd.read_csv('Precip.csv')
precipitation

In [ ]:
# Import necessaru libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
Precipitation=precipitation.drop('SNOW', axis=1)
Precipitation

In [ ]:
cormat=Precipitation.corr()
round(cormat,2)

In [ ]:
Precipitation.set_index('DATE')

In [ ]:
#make correlation heatmap
import seaborn as sns
sns.heatmap(Precipitation.corr(),annot=True)
plt.title('Feature correlation heatmap')
# Set the parameter for the size of the plot we want
plt.rcParams["figure.figsize"] = (20, 15)

In [ ]:
#Unit Conversion
Precipitation['Humidity']=Precipitation['Humidity'].apply(lambda x: float(x))
Precipitation['Humidity'] = Precipitation['Humidity'] * 0.01
Precipitation['Humidity'].round(2)
Precipitation['Precipitation'] = Precipitation['Precipitation'].apply(lambda x: round(x*25.4,2))
Precipitation['TMAX']= Precipitation['TMAX'].apply(lambda x: round((x-32)*(5/9)))
Precipitation['TMIN']= Precipitation['TMIN'].apply(lambda x: round((x-32)*(5/9)))
Precipitation['TAVG']= Precipitation['TAVG'].apply(lambda x: round((x-32)*(5/9)))
Precipitation

In [ ]:
#Data processing
p = Precipitation
p1 = p[p.isna().any(axis=1)]
p['TMIN'] = p['TMIN'].fillna(17.94)
Pfinal=p.loc[~(p['Precipitation']==0)]

In [ ]:
#log transformation
Pfinal['Precipitation']=np.log(p['Precipitation'])
Pfinal['Humidity']=np.log(p['Humidity'])
Pfinal['SLP'] = np.log(p['SLP'])
Pfinal['SP'] = np.log(p['SP'])
Pfinal['TMAX'] = np.log(p['TMAX'])
Pfinal['TMIN'] = np.log(p['TMIN'])
Pfinal['TAVG'] = np.log(p['TAVG'])
Pfinal['WD'] = np.log(p['WD'])
Pfinal['WDF2'] = np.log(p['WDF2'])
Pfinal['WS'] = np.log(p['WS'])
Pfinal['WSF2'] = np.log(p['WSF2'])
Pfinal

In [ ]:
# Importing necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

# Importing metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
sns.scatterplot(x="WSF2", y="Precipitation", data=Pfinal)
sns.lmplot(x="WSF2", y="Precipitation", data=Pfinal)
sns.lmplot(x="Humidity", y="Precipitation", data=Pfinal)
sns.lmplot(x="TMIN", y="Precipitation", data=Pfinal)

In [ ]:
Pfinal.set_index('DATE')

# Random Forest Regression

In [ ]:
#define input and output
X = Pfinal.loc[:,['LATITUDE','LONGITUDE','Humidity','SLP','SP','TMAX','TAVG','WSF2','WDF2','TMIN','WD','WS']]
y = Pfinal.loc[:,'Precipitation']

X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X, y, test_size=0.33, random_state=43)

In [ ]:
# Standardization
mean, std = X_train_df.mean(), X_train_df.std()

X_train_df = (X_train_df - mean)/std
X_test_df  = (X_test_df - mean)/std

X_train = X_train_df.to_numpy()
y_train = y_train_df.to_numpy()
X_test = X_test_df.to_numpy()
y_test = y_test_df.to_numpy()

print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

In [ ]:
# Created simple reporting function
def clf_performance(classifier, model_name):
    print(model_name)
    print('Best Score: ' + str(classifier.best_score_))
    print('Best Parameters: ' + str(classifier.best_params_))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rf = RandomForestRegressor()
param_grid =  {'n_estimators': [200,400,600,800,1000], 
                                  'bootstrap': [True,False],
                                  'max_depth': [10,20,30,40],
                                  'max_features': ['auto','sqrt'],
                                  'min_samples_leaf': [4,8,12,16,20],
                                  'min_samples_split': [5,10,15,25]}
                                  
rf_par = RandomizedSearchCV(estimator=rf, param_distributions = param_grid, n_iter = 10, cv = 200, verbose = 2,random_state = 100)
best_rf = rf_par.fit(X_train,y_train)
outputrf = best_rf.predict(X_test)
clf_performance(best_rf,'Random Forest')

In [ ]:
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
print('MAE:', mean_absolute_error(y_test, outputrf))
print('RMSE:', np.sqrt(mean_squared_error(y_test, outputrf)))
print('R2_Score:', r2_score(y_test, outputrf))

In [ ]:
fig = plt.figure(figsize=(8, 4))
plt.scatter(y_test, outputrf,color='paleturquoise', linewidths=2, edgecolors='k')
plt.xlabel('True Precipitation') 
plt.ylabel('Predicted Precipitation') 
plt.title('Random Forest Regression Prediction Performance') 
plt.grid()
plt.xlim(plt.xlim(0,6))
plt.ylim(plt.ylim(0,4))

m,b = np.polyfit(y_test,outputrf,1)
x = np.arange(y_test.min(),y_test.max(),5.5)
plt.plot(x,m*x+b)

In [ ]:
#creating time series plot
Pfinal_prep = Pfinal.set_index('DATE')
Pfinal_prep['Precipitation'].plot(figsize=(12,5),use_index = True)

In [ ]:
Pfinal_prep['Precipitation'].plot(figsize=(12,5),use_index = True)

In [ ]:
from matplotlib import pyplot
pyplot.plot(y_test, label='Expected')
pyplot.plot(outputrf, label='Predicted')
pyplot.legend()
pyplot.show()

In [ ]:
def plot_feature_importance(importance, names):
    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    #Define size of bar plot
    plt.figure(figsize=(10,8))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title('FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')

In [ ]:
plot_feature_importance(rf.feature_importances_, X.columns)

# XGBoost Model

In [ ]:
params_xgb = {'objective': 'reg:squarederror',
              #'base_score': 0.5,     # chosen as median of validation set
              'n_estimators': 800,  # number of trees to use
              #'learning_rate': 0.01, 
              'max_depth': 10,       # how many levels are in each tree
              #'subsample': 1,
              #'colsample_bytree': 0.8,
              # REGULARIZATION  alpha (L2) and lambda (L1)
              'reg_alpha': 0,
              'reg_lambda': 1,}

In [ ]:
model = XGBRegressor(**params_xgb)
model.fit(X_train, y_train)
rf_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
print('MAE:', mean_absolute_error(y_test, rf_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, rf_pred)))
print('R2_Score:', r2_score(y_test, rf_pred))

In [ ]:
def plot_feature_importance(importance, names):
    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    #Define size of bar plot
    plt.figure(figsize=(10,8))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title('FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')
plot_feature_importance(model.feature_importances_, X.columns)

In [ ]:
fig = plt.figure(figsize=(8, 4))
plt.scatter(y_test, rf_pred,color='paleturquoise', linewidths=2, edgecolors='k')
plt.xlabel('True Precipitation') 
plt.ylabel('Predicted Precipitation') 
plt.title('XGBoost Prediction Performance') 
plt.grid()
plt.xlim(plt.xlim(0,6))
plt.ylim(plt.ylim(0,4))

m,b = np.polyfit(y_test,rf_pred,1)
x = np.arange(y_test.min(),y_test.max(),5.5)
plt.plot(x,m*x+b)

In [ ]:
Pfinal_prep = Pfinal.set_index('DATE')
Pfinal_prep['Precipitation'].plot(figsize=(12,5),use_index = True)
from matplotlib import pyplot
pyplot.plot(y_test, label='Expected')
pyplot.plot(rf_pred, label='Predicted')
pyplot.legend()
pyplot.show()